In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# Importacion de librerias

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore")

import argparse
import tensorflow as tf
from IPython.display import display, clear_output
from random import randint

# Configuraciones de TensorFlow

In [ ]:
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True

### Modo eagerly

In [ ]:
#Comprobar que estoy ejecutandome en modo eagerly
tf.executing_eagerly()

# Carga de Datos

In [ ]:
root_path = "/home/jefelitman/DataSets/ucf101/split_1"
root_path

In [ ]:
batch_size = 32
size = [112,112]
frames = 16
canales = 3

In [ ]:
import sys
sys.path.append('../')

from Datasets_utils.DatasetsLoader import VideoDataGenerator

dataset = VideoDataGenerator(root_path,batch_size,True)

# Red Neuronal

In [ ]:
from LTC import LTC

Sesion = tf.Session(config=config)

video_shape = [frames]+size+[canales]

ltc = LTC(101,batch_size,video_shape)

ltc.enable_training()

In [ ]:
ltc.inicializar_modelo(0.003,0.5)
init = tf.global_variables_initializer()
Sesion.run(init)

In [ ]:
#Como es primera ver corriendo este modelo entonces no debo cargar nada
saver = tf.train.Saver()
step = 1
train_epoch = 1
test_epoch = 1
train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []
while True:
    #Primero voy salvando el modelo de 40000 en 40000
    if step % 40000 == 0:
        saved_path = saver.save(Sesion, '../Checkpoints/LTC_112x112x16_rgb')
        print('Modelo salvado en: {}'.format(saved_path))
        
    if step == 80000 or step == 125000:
        ltc.lr = ltc.lr * 0.1
    
    if step % 10 == 0:
        clear_output(wait=True)
        print(" Step: ",step)
        print("Train Epoch: ",train_epoch, " Train batch: ",dataset.train_batch_index+1,"/",dataset.train_batches)
        print("Train_Loss: ",train_loss[-1]," Train_Acuraccy: ",train_accuracy[-1])
        print("Test Epoch: ",test_epoch," Test batch: ",dataset.test_batch_index+1,"/",dataset.test_batches)
        print("Test_Loss: ",test_loss[-1]," Test_Acuraccy: ",test_accuracy[-1])
    
    if dataset.train_batch_index == dataset.train_batches:
        train_epoch += 1
        
    if dataset.test_batch_index == dataset.test_batches:
        test_epoch += 1
    
    #Entreno con datos de entrenamiento
    batch, labels = dataset.get_next_train_batch(size,frames,canales)
    Sesion.run(ltc.entrenar,feed_dict={ltc.x: batch, ltc.y: labels})
    train_loss.append(Sesion.run(ltc.perdida,feed_dict={ltc.x: batch, ltc.y: labels}))
    train_accuracy.append(Sesion.run(ltc.precision,feed_dict={ltc.x: batch, ltc.y: labels}))
    
    #Pruebo con datos de test
    batch, labels = dataset.get_next_test_batch(size,frames,canales)
    test_loss.append(Sesion.run(ltc.perdida,feed_dict={ltc.x: batch, ltc.y: labels}))
    test_accuracy.append(Sesion.run(ltc.precision,feed_dict={ltc.x: batch, ltc.y: labels}))
    
    step += 1

In [ ]:
tf.train.get_or_create_global_step().eval(Sesion)